# License Plate Detection Model Training with YOLOv11

This notebook trains a YOLOv11 model to detect license plates in images using the plateRecignation dataset.


In [1]:
# Install required packages
%pip install ultralytics pillow opencv-python -q


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Setup and Import Libraries


In [3]:
import os
import shutil
import xml.etree.ElementTree as ET
from pathlib import Path
import random
from ultralytics import YOLO
import cv2

# Set random seed for reproducibility
random.seed(42)

# Define paths
dataset_root = Path("plateRecignation")
train_dir = dataset_root / "train"
test_dir = dataset_root / "test"

print("Libraries imported successfully!")
print(f"Train images: {len(list(train_dir.glob('*.jpg')))}")
print(f"Test images: {len(list(test_dir.glob('*.jpg')))}")


Libraries imported successfully!
Train images: 567
Test images: 142


## 2. Convert Pascal VOC XML to YOLO Format


In [4]:
def convert_xml_to_yolo(xml_path, output_dir):
    """
    Convert Pascal VOC XML annotation to YOLO format.
    YOLO format: class_id center_x center_y width height (normalized 0-1)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Get image dimensions
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    # Create output label file
    label_file = output_dir / (xml_path.stem + '.txt')
    
    with open(label_file, 'w') as f:
        # Find all objects (license plates)
        for obj in root.findall('object'):
            # Class name is 'LP' (License Plate)
            class_name = obj.find('name').text
            
            # For YOLO, we use class_id 0 for license plate (single class)
            class_id = 0
            
            # Get bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            # Convert to YOLO format (normalized center coordinates and dimensions)
            center_x = (xmin + xmax) / 2.0 / img_width
            center_y = (ymin + ymax) / 2.0 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            # Write to file
            f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
    
    return label_file.exists()

print("Conversion function defined!")


Conversion function defined!


In [5]:
# Create YOLO dataset structure
yolo_dataset = Path("plateRecignation_yolo")
for split in ["train", "valid"]:
    (yolo_dataset / split / "images").mkdir(parents=True, exist_ok=True)
    (yolo_dataset / split / "labels").mkdir(parents=True, exist_ok=True)

print("YOLO dataset directories created!")


YOLO dataset directories created!


In [6]:
# Get all training images
train_images = list(train_dir.glob("*.jpg"))
print(f"Total training images: {len(train_images)}")

# Shuffle and split: 80% train, 20% validation
random.shuffle(train_images)
split_idx = int(0.8 * len(train_images))
train_split = train_images[:split_idx]
val_split = train_images[split_idx:]

print(f"Train split: {len(train_split)}, Validation split: {len(val_split)}")


Total training images: 567
Train split: 453, Validation split: 114


In [7]:
# Process training images
print("Processing training images...")
for img_path in train_split:
    # Copy image
    dest_img = yolo_dataset / "train" / "images" / img_path.name
    shutil.copy2(img_path, dest_img)
    
    # Convert and save label
    xml_path = train_dir / (img_path.stem + ".xml")
    if xml_path.exists():
        convert_xml_to_yolo(xml_path, yolo_dataset / "train" / "labels")
    else:
        print(f"Warning: XML not found for {img_path.name}")

print(f"Processed {len(train_split)} training images")


Processing training images...
Processed 453 training images


In [8]:
# Process validation images
print("Processing validation images...")
for img_path in val_split:
    # Copy image
    dest_img = yolo_dataset / "valid" / "images" / img_path.name
    shutil.copy2(img_path, dest_img)
    
    # Convert and save label
    xml_path = train_dir / (img_path.stem + ".xml")
    if xml_path.exists():
        convert_xml_to_yolo(xml_path, yolo_dataset / "valid" / "labels")
    else:
        print(f"Warning: XML not found for {img_path.name}")

print(f"Processed {len(val_split)} validation images")


Processing validation images...
Processed 114 validation images


## 4. Create data.yaml Configuration File


In [9]:
# Create data.yaml file for YOLO
yaml_content = """train: train/images
val: valid/images

nc: 1
names: ['license_plate']
"""

yaml_path = yolo_dataset / "data.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("data.yaml created!")
print(yaml_content)


data.yaml created!
train: train/images
val: valid/images

nc: 1
names: ['license_plate']



## 5. Initialize and Train YOLOv11 Model


In [10]:
# Check CUDA availability
import torch
if torch.cuda.is_available():
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    device = 0  # Use first GPU
else:
    print("CUDA is not available. Training will use CPU (slower).")
    device = 'cpu'

# Initialize YOLOv11 model
# Options: 'yolo11n.pt' (nano - fastest), 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt', 'yolo11x.pt' (most accurate)
model = YOLO('yolo11n.pt')  # Start with nano for faster training

print("Model initialized!")


CUDA is available! Using GPU: NVIDIA GeForce GTX 1650 with Max-Q Design
CUDA Version: 11.8
Model initialized!


In [11]:
# Train the model
results = model.train(
    data=str(yaml_path),
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size
    batch=16,   # Batch size (adjust based on your GPU memory)
    device=device,  # Use CUDA if available, otherwise CPU
    name='plate_detection_yolo11',  # Project name
    project='runs/detect',  # Save directory
    patience=15,  # Early stopping patience
    save=True,
    plots=True,
    verbose=True
)

print("Training completed!")


New https://pypi.org/project/ultralytics/8.3.234 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=plateRecignation_yolo\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=plate_

## 6. Evaluate the Model


In [12]:
# Find the latest model automatically
runs_dir = Path("runs/detect")
model_dirs = sorted([d for d in runs_dir.glob("plate_detection_yolo11*") if d.is_dir()], 
                    key=lambda x: x.stat().st_mtime, reverse=True)

if model_dirs:
    best_model_path = model_dirs[0] / "weights" / "best.pt"
    if best_model_path.exists():
        print(f"Loading model from: {best_model_path}")
        model = YOLO(str(best_model_path))
        
        # Evaluate on validation set
        metrics = model.val(data=str(yaml_path), split='val')
        print(f"\nValidation Results:")
        print(f"mAP50: {metrics.box.map50:.4f}")
        print(f"mAP50-95: {metrics.box.map:.4f}")
        print(f"Precision: {metrics.box.mp:.4f}")
        print(f"Recall: {metrics.box.mr:.4f}")
    else:
        print(f"Error: best.pt not found in {model_dirs[0]}")
        print("Available files:", list((model_dirs[0] / "weights").glob("*.pt")))
else:
    print("Error: No model directories found. Please train the model first.")


Loading model from: runs\detect\plate_detection_yolo11\weights\best.pt
Ultralytics 8.3.233  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 369.696.4 MB/s, size: 211.4 KB)
val: Scanning C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\valid\labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 114/114 157.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.4s/it 11.2s0.3s
                   all        114        114      0.991          1      0.995      0.804
Speed: 3.6ms preprocess, 9.8ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\license-plate-recognition\runs\detect\val2

Validation Results:
mAP50: 0.995

## 7. Test Detection on Sample Images


In [13]:
# Test on a few sample images from validation set
import matplotlib.pyplot as plt
import numpy as np

# Get a few validation images
val_images = list((yolo_dataset / "valid" / "images").glob("*.jpg"))[:3]

fig, axes = plt.subplots(1, len(val_images), figsize=(15, 5))
if len(val_images) == 1:
    axes = [axes]

for idx, img_path in enumerate(val_images):
    # Run inference
    results = model(str(img_path))
    
    # Plot results
    annotated_img = results[0].plot()
    axes[idx].imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
    axes[idx].axis('off')
    axes[idx].set_title(f"Image: {img_path.name}")

plt.tight_layout()
plt.show()

print("Sample predictions displayed!")



image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\valid\images\148.jpg: 480x640 1 license_plate, 84.7ms
Speed: 4.5ms preprocess, 84.7ms inference, 10.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\valid\images\158.jpg: 480x640 1 license_plate, 20.2ms
Speed: 4.4ms preprocess, 20.2ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\valid\images\167.jpg: 640x480 1 license_plate, 69.1ms
Speed: 4.0ms preprocess, 69.1ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 480)


<Figure size 1500x500 with 3 Axes>

Sample predictions displayed!


## 8. Test on Test Set Images


In [14]:
# Process test images for evaluation
test_images = list(test_dir.glob("*.jpg"))
print(f"Total test images: {len(test_images)}")

# Create test directory structure
(yolo_dataset / "test" / "images").mkdir(parents=True, exist_ok=True)
(yolo_dataset / "test" / "labels").mkdir(parents=True, exist_ok=True)

# Process test images
print("Processing test images...")
for img_path in test_images:
    # Copy image
    dest_img = yolo_dataset / "test" / "images" / img_path.name
    shutil.copy2(img_path, dest_img)
    
    # Convert and save label
    xml_path = test_dir / (img_path.stem + ".xml")
    if xml_path.exists():
        convert_xml_to_yolo(xml_path, yolo_dataset / "test" / "labels")

print(f"Processed {len(test_images)} test images")


Total test images: 142
Processing test images...
Processed 142 test images


In [15]:
# Update data.yaml to include test set
yaml_content = """train: train/images
val: valid/images
test: test/images

nc: 1
names: ['license_plate']
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("data.yaml updated with test set!")


data.yaml updated with test set!


In [16]:
# Evaluate on test set
if model_dirs and (model_dirs[0] / "weights" / "best.pt").exists():
    model = YOLO(str(model_dirs[0] / "weights" / "best.pt"))
    
    # Evaluate on test set
    test_metrics = model.val(data=str(yaml_path), split='test')
    print(f"\nTest Set Results:")
    print(f"mAP50: {test_metrics.box.map50:.4f}")
    print(f"mAP50-95: {test_metrics.box.map:.4f}")
    print(f"Precision: {test_metrics.box.mp:.4f}")
    print(f"Recall: {test_metrics.box.mr:.4f}")
else:
    print("Model not found. Please train the model first.")


Ultralytics 8.3.233  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 11.36.7 MB/s, size: 124.2 KB)
val: Scanning C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\test\labels... 142 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 142/142 245.9it/s 0.6s0.1s
val: New cache created: C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\plateRecignation_yolo\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.9it/s 3.1s0.3ss
                   all        142        142      0.993          1      0.995      0.765
Speed: 2.6ms preprocess, 6.9ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\license-plate-recognit

## 9. Save Model for Inference

The trained model is saved at:
- **Best model**: `runs/detect/plate_detection_yolo11*/weights/best.pt`
- **Last model**: `runs/detect/plate_detection_yolo11*/weights/last.pt`

You can use the model for inference like this:
```python
from ultralytics import YOLO

# Load the trained model
model = YOLO('runs/detect/plate_detection_yolo11*/weights/best.pt')

# Run inference on an image
results = model('path/to/image.jpg')

# Display results
results[0].show()
```
